In [1]:
from datetime import datetime
from pathlib import Path
import gsw
import numpy as np
import pandas as pd
import xarray as xr
import pyarrow.parquet as pq
import requests as rq
import os
from urllib.parse import urljoin, urlencode
import argo_tools as at
import argopy
from argopy import DataFetcher as ArgoDataFetcher
local_gdac = '/vortexfs1/share/boom/data/gdac_snapshot/202403-ArgoData'
#Path.mkdir('/vortexfs1/share/boom/projects/n2o/pq')
outdir = '/vortexfs1/share/boom/projects/n2o/pq'
argopy.set_options(mode='expert',src='gdac',ftp=local_gdac)
from pprint import pprint
import warnings
import glob
import psutil

In [2]:
wmos, df2, wmo_fp = at.argo_gdac()

/vortexfs1/home/enrico.milanese/projects/ARGO/nc2parquet/argo_tools.py:47: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  gdac_index = pd.read_csv(gdac_path,delimiter=',',header=8,parse_dates=['date','date_update'],


In [3]:
pprint(wmo_fp)

array(['aoml/1900722/', 'aoml/1901378/', 'aoml/1901379/', ...,
       'meds/4902553/', 'meds/4902554/', 'meds/4902555/'], dtype=object)


#### Importing iterators from [`itertools`](https://docs.python.org/3/library/itertools.html)
* `islice` returns selected elements from iterable
* `batched` split the iterable object into tuples of prescribed length _n_ (if `length(iterable)%n~=0`, the last tuple is shorter than _n_).

In [4]:
import sys
from itertools import islice

if sys.version_info >= (3, 12):
    from itertools import batched
else:
    try:
        from more_itertools import batched
    except ImportError:
        def batched(iterable, chunk_size):
            iterator = iter(iterable)
            while chunk := tuple(islice(iterator, chunk_size)):
                yield chunk

In [5]:
VARS = ['JULD','CYCLE_NUMBER','PLATFORM_NUMBER','LATITUDE','LONGITUDE',
 'PRES',
 'PRES_QC',
 'PRES_ADJUSTED',
 'PRES_ADJUSTED_QC',
 'PRES_ADJUSTED_ERROR',
 'TEMP',
 'TEMP_QC',
 'TEMP_dPRES',
 'TEMP_ADJUSTED',
 'TEMP_ADJUSTED_QC',
 'TEMP_ADJUSTED_ERROR',
 'PSAL',
 'PSAL_QC',
 'PSAL_dPRES',
 'PSAL_ADJUSTED',
 'PSAL_ADJUSTED_QC',
 'PSAL_ADJUSTED_ERROR',
 'DOXY',
 'DOXY_QC',
 'DOXY_dPRES',
 'DOXY_ADJUSTED',
 'DOXY_ADJUSTED_QC',
 'DOXY_ADJUSTED_ERROR',
 'CHLA',
 'CHLA_QC',
 'CHLA_dPRES',
 'CHLA_ADJUSTED',
 'CHLA_ADJUSTED_QC',
 'CHLA_ADJUSTED_ERROR',
 'BBP700',
 'BBP700_QC',
 'BBP700_dPRES',
 'BBP700_ADJUSTED',
 'BBP700_ADJUSTED_QC',
 'BBP700_ADJUSTED_ERROR',
 'CDOM',
 'CDOM_QC',
 'CDOM_dPRES',
 'CDOM_ADJUSTED',
 'CDOM_ADJUSTED_QC',
 'CDOM_ADJUSTED_ERROR',
 'PH_IN_SITU_TOTAL',
 'PH_IN_SITU_TOTAL_QC',
 'PH_IN_SITU_TOTAL_dPRES',
 'PH_IN_SITU_TOTAL_ADJUSTED',
 'PH_IN_SITU_TOTAL_ADJUSTED_QC',
 'PH_IN_SITU_TOTAL_ADJUSTED_ERROR',
 'NITRATE',
 'NITRATE_QC',
 'NITRATE_dPRES',
 'NITRATE_ADJUSTED',
 'NITRATE_ADJUSTED_QC',
 'NITRATE_ADJUSTED_ERROR']

### Parallel 

In [6]:
import multiprocessing

def xr2pqt(rank,files_list):
    df_list = []
    df_memory = 0
    counter = 0
    rank_str = "#" + str(rank) + ": "
    nb_files = len(files_list)
    for argo_file in files_list:
        counter += 1
        if counter%10==0:
            print(rank_str + "processing file " + str(counter) + " of " + str(nb_files))
            
        try:
            ds = xr.load_dataset(argo_file, engine="argo") #loading into memory the profile
        except:
            print(rank_str + 'Failed on ' + str(argo_file))
        
        invars = list(set(VARS) & set(list(ds.data_vars)))
        df = ds[invars].to_dataframe()
        df_memory += df.memory_usage().sum()/(10**9) # tracking memory usage (in GB)
        df_list.append(df)

        # print(rank_str + "{:.3f}".format(df_memory))

    # store to parquet once a large enough dataframe has been created
    
    print(rank_str + "Storing to parquet...")
    print(rank_str + "Filesize: " + "{:.2f}".format(df_memory) + " GB")
    
    df_list = pd.concat(df_list)

    # root_storage = "/vortexfs1/home/enrico.milanese/projects/ARGO/nc2parquet/PQT_test/test_parquet_"
    root_storage = "/vortexfs1/share/boom/data/nc2pqt_test/"
    parquet_filename = root_storage + "test_parquet_" + str(rank) + ".parquet"
    df_list.to_parquet(parquet_filename)
    print(rank_str + str(parquet_filename) + " stored.")

    # print("Storing to csv...")
    # csv_filename = "/vortexfs1/home/enrico.milanese/projects/ARGO/nc2parquet/PQT_test/test_parquet_" + str(pqt_files) + ".csv"
    # df_list.to_csv(csv_filename)
    # print(str(csv_filename) + " stored.")
    df_list = []
    df_memory = 0
    
############################################################################################################

def poolParams(flist):
    size_flist = []
    for f in flist:
        size_flist.append( os.path.getsize(f)/1024**2 ) #size in MB
    
    size_tot = sum(size_flist)
    NPROC = int(np.ceil(size_tot/300)) # Empirically, 300 MB of .nc files seems a good trade-off
    size_per_proc = size_tot/NPROC

    print('')
    print('Size per processor (MB)')
    print(size_per_proc)
    print('')
    
    ids_sort = np.argsort(np.array(size_flist))
    
    chunks_ids = []
    x = np.copy(ids_sort)
    
    for j in range(NPROC):
        chunk_ids = []
        chunk_size = 0
        while ((chunk_size<size_per_proc) and (len(x) > 0)):
            if len(chunk_ids)%2 == 0:
                chunk_ids.append(x[-1])
                x = x[:-1]
            else:
                chunk_ids.append(x[0])
                x = x[1:]
            chunk_size = sum(np.asarray(size_flist)[chunk_ids])
        print(chunk_size)
        chunks_ids.append(chunk_ids)
    
    if len(x) > 0:
        warnings.warn(str(len(x)) + " files have not been assigned to a processor.")
    
    print('')
    chunks=[]
    skip_proc = 0
    total_memory = 0
    for j,chunk_ids in enumerate(chunks_ids):
        print('Size in processor ' + str(j) + ' (MB):')
        size_proc = sum(np.asarray(size_flist)[chunk_ids])
        total_memory += size_proc
        print(size_proc)
        if size_proc == 0:
            skip_proc += 1
            continue
        chunk = [flist[k] for k in chunk_ids]
        chunks.append(chunk)

    NPROC -= skip_proc
        
    print('')
    print("Using " + str(NPROC) + " processors")
    
    return NPROC, chunks

############################################################################################################

# wmos, df2, wmo_fp = at.argo_gdac()

# flist = []
# for line in wmo_fp:  
#         if '6902958' in line:
#             print('skipping 6902958')
#         else:
#             fn = (line.split('/')[1] + "_Sprof.nc")
#             fpath = Path(local_gdac) / "dac" / line / fn
#             if os.path.isfile(fpath):
#                 flist.append(str(fpath))
#             else:
#                 warnings.warn("File " + str(fpath) + " not found, skipping it.")

# print(flist)

# flist = glob.glob("/vortexfs1/home/enrico.milanese/projects/ARGO/nc2parquet/GDAC/pub/dac/aoml/*/*_Sprof.nc")
flist = glob.glob("/vortexfs1/share/boom/data/gdac_snapshot/202403-ArgoData/dac/aoml/*/*_Sprof.nc")

print("Processing " + str(len(flist)) + " files.")

NPROC, chunks = poolParams(flist)

pool_obj = multiprocessing.Pool(processes=NPROC)
pool_obj.starmap(xr2pqt, [(rank, chunk) for rank, chunk in enumerate(chunks)] )
pool_obj.close()


Processing 793 files.

Size per processor (MB)
277.2022272348404

277.41802978515625
280.36266231536865
277.52312088012695
280.22032356262207
277.9414978027344
278.5088586807251
278.4192123413086
267.22411251068115

Size in processor 0 (MB):
277.41802978515625
Size in processor 1 (MB):
280.36266231536865
Size in processor 2 (MB):
277.52312088012695
Size in processor 3 (MB):
280.22032356262207
Size in processor 4 (MB):
277.9414978027344
Size in processor 5 (MB):
278.5088586807251
Size in processor 6 (MB):
278.4192123413086
Size in processor 7 (MB):
267.22411251068115

Using 8 processors
#7: processing file 10 of 102
#6: processing file 10 of 106
#5: processing file 10 of 103
#1: processing file 10 of 97
#2: processing file 10 of 104
#3: processing file 10 of 105
#4: processing file 10 of 103
#1: processing file 20 of 97
#6: processing file 20 of 106
#3: processing file 20 of 105
#7: processing file 20 of 102
#5: processing file 20 of 103
#2: processing file 20 of 104
#4: processing file

### Test direct subsetting from parquet directory

In [ ]:
#t = pq.read_table(Path(outdir),filters=[('PRES', '<', 20),('LATITUDE', '<', 21.1),('LATITUDE', '>', 21)])
t = pq.read_table(Path(outdir),filters=[('PLATFORM_NUMBER', '==', 1902304)])
df = t.to_pandas()
df
#ds = xr.Dataset.from_dataframe(df)
#ds

### Example loading Sprof from snapshot
```
ds = xr.load_dataset('/vortexfs1/share/boom/data/gdac_snapshot/202403-ArgoData/dac/aoml/1902304/1902304_Sprof.nc')
df = ds.to_dataframe()
```

In [ ]:

from dotenv import load_dotenv
load_dotenv()
import os
from pyarrow import fs
s3 = fs.S3FileSystem(region='us-east-1')


In [ ]:
s3

In [ ]:
import pyarrow as pa
import pyarrow.parquet as pq
from pyarrow import Table

ds = xr.load_dataset('/vortexfs1/share/boom/data/gdac_snapshot/202403-ArgoData/dac/aoml/1902304/1902304_Sprof.nc',engine="argo")
df = ds[['DOXY','PRES','NITRATE','PLATFORM_NUMBER']].to_dataframe()

s3_filepath = 'data.parquet'

pq.write_to_dataset(
    Table.from_pandas(df),
    s3_filepath,
    filesystem=s3,
    use_dictionary=True,
    compression="snappy",
    version="2.4",
)



### single threaded

In [ ]:

CHUNK_SZ = 100
VARS = ['JULD','LATITUDE','LONGITUDE','PRES','PRES_ADJUSTED','DOXY_ADJUSTED','DOXY_ADJUSTED_QC','NITRATE','NITRATE_ADJUSTED','PSAL','TEMP','CYCLE_NUMBER','PLATFORM_NUMBER']
for chunk in batched(wmo_fp,CHUNK_SZ):
    dflist = []
    for line in chunk:  
        fn = (line.split('/')[1] + "_Sprof.nc")
        fpath = Path(local_gdac) / "dac" / line / fn
        try:
            ds = xr.load_dataset(fpath)
        except:
            print(fpath)
        invars = list(set(VARS) & set(list(ds.data_vars)))
        df = ds[invars].to_dataframe()
        dflist.append(df)
    print(fpath)
    df = pd.concat(dflist)
    df.to_parquet('pq/test' + line.split('/')[1] + ".parquet",coerce_timestamps='us',allow_truncated_timestamps=True)
        
        